In [4]:
import numpy as np
from scipy.optimize import linprog
from bac import solve_mip


# Problem parameters
num_facilities = 3
num_customers = 5
num_scenarios = 1000

# Fixed costs for opening facilities
fixed_costs = np.array([100000, 120000, 90000])

# Transportation costs from facilities to customers
trans_costs = np.array([
    [10, 15, 20, 25, 30],
    [12, 14, 18, 22, 26],
    [8, 16, 24, 32, 40]
])

# Facility capacities
capacities = np.array([5000, 6000, 4500])

# Generate random demand scenarios
np.random.seed(42)
demand_mean = 1000
demand_std = 200
demand_scenarios = np.random.normal(demand_mean, demand_std, (num_scenarios, num_customers))
demand_scenarios = np.clip(demand_scenarios, 0, None)  # Ensure non-negative demand

# Penalty parameter for the algorithm
rho = 1.0

# Number of iterations
max_iterations = 100

# Convergence tolerance
tolerance = 1e-4

# Initialize decision variables and dual variables
x = np.zeros((num_scenarios, num_facilities, num_customers))
y = np.zeros((num_scenarios, num_facilities))  # Binary decision for opening facilities
lambda_x = np.zeros((num_scenarios, num_facilities, num_customers))
lambda_y = np.zeros((num_scenarios, num_facilities))

# Helper function to solve the subproblem for each scenario
def solve_subproblem(demands, fixed_costs, trans_costs, capacities, lambda_x, lambda_y, x_avg, y_avg, rho, s):
    num_facilities, num_customers = trans_costs.shape
    num_vars = num_facilities + num_facilities * num_customers

    # Objective function coefficients
    c = np.zeros(num_vars)
    c[:num_facilities] = fixed_costs
    c[num_facilities:] = trans_costs.flatten()

    # Add augmented Lagrangian terms to the objective
    for i in range(num_facilities):
        c[i] += lambda_y[i] * 1 + 0.5 * rho * (1 - y_avg[i])**2
        for j in range(num_customers):
            idx = num_facilities + i*num_customers + j
            c[idx] += lambda_x[i, j] * 1 + 0.5 * rho * (1 - x_avg[i, j])**2

    # Constraints
    A_ub = []
    b_ub = []

    # Capacity constraints
    for i in range(num_facilities):
        row = np.zeros(num_vars)
        row[i] = -capacities[i]
        row[num_facilities + i*num_customers : num_facilities + (i+1)*num_customers] = 1
        A_ub.append(row)
        b_ub.append(0)

    # Demand satisfaction constraints
    A_eq = []
    b_eq = []
    for j in range(num_customers):
        row = np.zeros(num_vars)
        for i in range(num_facilities):
            row[num_facilities + i*num_customers + j] = 1
        A_eq.append(row)
        b_eq.append(demands[j])

    # Convert to numpy arrays
    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)

    # Bounds
    bounds = [(0, 1)] * num_facilities + [(0, None)] * (num_vars - num_facilities)

    int_vars = (np.zeros(len(c), dtype=int))
    int_vars[:num_facilities] = 1
    int_vars = int_vars.tolist()


    # Solve the problem
    result = solve_mip(c, A_ub, b_ub, A_eq, b_eq, bounds, int_vars, minimize=True)

    # Extract results
    y = result[0][:num_facilities]
    x = result[0][num_facilities:].reshape(num_facilities, num_customers)
    obj = result[1]

    
    return x, y, obj

# Progressive Hedging Algorithm
for iteration in range(max_iterations):
    x_old = x.copy()
    y_old = y.copy()
    
    # Compute averages
    x_avg = np.mean(x, axis=0)
    y_avg = np.mean(y, axis=0)
    
    # Solve subproblems for each scenario
    obj_list = []
    for s in range(num_scenarios):
        x[s], y[s], obj = solve_subproblem(demand_scenarios[s], fixed_costs, trans_costs, capacities, 
                                           lambda_x[s], lambda_y[s], x_avg, y_avg, rho, s)
        obj_list.append(obj)

    
    # Update dual variables
    for s in range(num_scenarios):
        lambda_x[s] += rho * (x[s] - x_avg)
        lambda_y[s] += rho * (y[s] - y_avg)
    
    # Check for convergence
    if np.max(np.abs(x - x_old)) < tolerance and np.max(np.abs(y - y_old)) < tolerance:
        print(f"Converged in {iteration + 1} iterations")
        break

    print(f"Iteration {iteration}, Avg Obj: {np.mean(obj_list)}")

# Display final decision variables
print("Facility opening decisions (y):")
print(np.round(y_avg))
print("Transportation decisions (x):")
print(np.round(x_avg))
print("Objective")



Iteration 0, Avg Obj: 436760.8062450729
Converged in 2 iterations
Facility opening decisions (y):
[1. 1. 1.]
Transportation decisions (x):
[[   0.   18.    0.    0.    0.]
 [1000.    0.    0.    0.    0.]
 [   0.  979. 1001. 1001. 1007.]]
Objective


In [5]:
991*12 + 1015*16 + 954*24 + 1026*32 + 994*40 + 24*20 +310000

434100